In [17]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [19]:
from sklearn.datasets import load_iris # Load Iris Data

iris = load_iris() # Creating pd DataFrames

iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
target_df = pd.DataFrame(data= iris.target, columns= ['species'])

def converter(specie):
    if specie == 0:
        return 'setosa'
    elif specie == 1:
        return 'versicolor'
    else:
        return 'virginica'

target_df['species'] = target_df['species'].apply(converter)# Concatenate the DataFrames
iris_df = pd.concat([iris_df, target_df], axis= 1)

In [20]:
# Converting Objects to Numerical dtype
iris_df.drop('species', axis= 1, inplace= True)
target_df = pd.DataFrame(columns= ['species'], data= iris.target)
iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
y= iris_df['sepal length (cm)']

# Splitting the Dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.13, random_state= 101)

# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X_train, y_train)

# Making Predictions
lr.predict(X_test)
pred = lr.predict(X_test)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))

mse = mean_squared_error(y_test, pred)

Mean Squared Error: 0.1123029571353145


In [5]:
shuffle = lambda x: np.random.shuffle(x)

In [21]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
X = X_test.values[:]
Yt_expected = y_test.values[:].reshape(-1, 1)
mse_target = 0.11232375

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('dataset/X.npy',X)
np.save('dataset/Y.npy',Yt_expected)

In [22]:
shuffle(W)
shuffle(b)
print(mean_squared_error(y_test, np.matmul(X_test, W.reshape(-1)) + b.reshape(-1)))

np.save('model_shuffled/W.npy',W)
np.save('model_shuffled/b.npy',b)

4.342399572275102


In [23]:
Y_expected = np.matmul(X_test, lr.coef_) + lr.intercept_

assert (Y_expected == lr.predict(X_test)).all()

In [24]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = mse_target
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

In [26]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]


In [27]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[
  {
    'publickey-1': '7740372084585987155484008265620943144342913409804205385745482832325610494802',
    'publickey-2': '13432246653756375231286144066765295473357154911140864565409792325468570091416',
    'MSE-Cap': '18406'
  },
  {
    'publickey-1': '3246617400318576376625010382666224414176861216974133155998074137567803868032',
    'publickey-2': '14826565776914610395183641281943720019980952665080130313463019842070299119071',
    'MSE-Cap': '18406'
  }
]


In [13]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  23.683279741029036
... quantized  65536
Mean Squared Error simulated:  23.68532014893487414
... quantized  65536
Circuit Outputs:
3880370
Proof took 18.778 s
ERROR: Invalid proof
An unexpected error occurred:

Error: Could not verify the proof
    at SimpleTaskDefinition.action (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/hardhat.config.js:246:26)
    at Environment._runTaskDefinition (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:217:14)
    at Environment.run (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:129:14)
    at main (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/cli/cli.ts:197:5)


In [29]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.11230295713531464
... quantized  18403
Mean Squared Error simulated:  0.11232375022889581895
... quantized  18406
Circuit Outputs:
18406
Proof took 19.356 s
INFO: OK!
Paying 0x2546BcD3c84621e976D8185a91A922aE77ECEc30
With balance
10605.6
Your Public Key: 
[
  19021983628265687208128175269911126266650427881307789728325305942159328916488n,
  7087564311867548326853970234010538242126275819924914853045245894144217480187n
]
Your Private Key: 
188869269427939293780456796238515871697913918280882814102873465580549025981n
Current Balance
10654.6


In [30]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[
  {
    'publickey-1': '7740372084585987155484008265620943144342913409804205385745482832325610494802',
    'publickey-2': '13432246653756375231286144066765295473357154911140864565409792325468570091416',
    'MSE-Cap': '18406'
  }
]


In [31]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]


In [ ]:
zkml add_bounty --amount 11.1 --keyfile ./out --walletprivatekey ./keys/.private_key --dataset ./dataset --settings ./settings.json